<a href="https://colab.research.google.com/github/ravi-prakash1907/Machine-Learning-for-Cyber-Security/blob/main/Labs/deepNN_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment-1 : House Price Prediction using Deep-Learning

In [90]:
## lib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from tensorflow import keras
from keras.models import Sequential 
from keras.layers import Dense

import requests

## Data Collection

In [91]:
## pulling dataset from my GitHub

In [92]:
# function on my gist: https://gist.github.com/bc3b75e60d321baca00102378122512d.git
def downloadCSV(fileURL, saveAs='downloaded.csv'):
  req = requests.get(fileURL)
  fileURLContent = req.content
  csv_file = open(saveAs, 'wb')
  
  csv_file.write(fileURLContent)
  csv_file.close()

In [93]:
dataLoc = "https://raw.githubusercontent.com/ravi-prakash1907/Machine-Learning-for-Cyber-Security/main/Labs/dataset/housepricedata.csv?token=AJGAAOBNEAI7J3AXQ53GLM3AX5NOU"

downloadCSV(dataLoc,'housepricedata.csv')

In [94]:
df = pd.read_csv("housepricedata.csv")

## Data Understanding

In [95]:
df.head()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,AboveMedianPrice
0,8450,7,5,856,2,1,3,8,0,548,1
1,9600,6,8,1262,2,0,3,6,1,460,1
2,11250,7,5,920,2,1,3,6,1,608,1
3,9550,7,5,756,1,0,3,7,1,642,0
4,14260,8,5,1145,2,1,4,9,1,836,1


In [96]:
## looking at the shape
df.shape

(1460, 11)

In [97]:
## looking for the detailed description of the dataset
df.describe()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,AboveMedianPrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,6.099315,5.575342,1057.429452,1.565068,0.382877,2.866438,6.517808,0.613014,472.980137,0.498630
std,9981.264932,1.382997,1.112799,438.705324,0.550916,0.502885,0.815778,1.625393,0.644666,213.804841,0.500169
min,1300.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000
25%,7553.500000,5.000000,5.000000,795.750000,1.000000,0.000000,2.000000,5.000000,0.000000,334.500000,0.000000
50%,9478.500000,6.000000,5.000000,991.500000,2.000000,0.000000,3.000000,6.000000,1.000000,480.000000,0.000000
75%,11601.500000,7.000000,6.000000,1298.250000,2.000000,1.000000,3.000000,7.000000,1.000000,576.000000,1.000000
max,215245.000000,10.000000,9.000000,6110.000000,3.000000,2.000000,8.000000,14.000000,3.000000,1418.000000,1.000000


**Hence, our aim is to predict the value of last column i.e. `AboveMedianPrice`**

## Data Preparation

In [176]:
## train - test split : 70:30
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[:-1]], df['AboveMedianPrice'], test_size=0.3)

In [177]:
## splitting the test set further in : test & validation set
X_valid, X_test, Y_valid, Y_test = train_test_split(X_test, Y_test, test_size=0.5)

In [178]:
## here traing data has 70%, while validation and testing have 15%-15% of the whole dataset

### Train-Test-Validation Data

In [179]:
# features
display(X_train.head(3))
display(X_test.head(3))
display(X_valid.head(3))

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
1146,11200,6,5,1298,2,0,3,5,1,403
754,7930,6,8,1026,1,0,3,5,0,440
1420,11700,6,6,708,2,1,3,7,1,776


,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
493,7931,5,6,1269,1,1,3,6,1,308
385,3182,8,5,1256,2,0,2,6,1,430
307,7920,6,7,319,1,0,3,6,0,0


,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
797,7677,5,5,773,1,0,2,4,0,240
1166,10475,8,5,1694,2,0,3,7,0,776
1173,18030,5,6,1598,3,0,3,12,1,0


In [180]:
# labels
display(Y_train.head(3))
display(Y_test.head(3))
display(Y_valid.head(3))

1146    1
754     0
1420    1
Name: AboveMedianPrice, dtype: int64

493    0
385    1
307    0
Name: AboveMedianPrice, dtype: int64

797     0
1166    1
1173    1
Name: AboveMedianPrice, dtype: int64

In [181]:
## normalization
min_max_scaler = preprocessing.MinMaxScaler()

X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)
X_valid = min_max_scaler.fit_transform(X_valid)

## Model Build

In [182]:
model = Sequential([
                    Dense(32, activation='relu', input_shape=(10,)),
                    Dense(32, activation='relu'),
                    Dense(1, activation='sigmoid')
])

In [183]:
model.compile(optimizer='sgd',
                loss='binary_crossentropy',
                metrics=['accuracy', 'mse'])

In [184]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 150 == 0: print('')
    print('.',end='')


early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)
history = model.fit(X_train, Y_train, epochs=1200, verbose=0, validation_data=(X_valid, Y_valid),# validation_split=0.1,
                    callbacks = [early_stop, PrintDot()] )
  
hist = pd.DataFrame(history.history)
hist['epoc'] = history.epoch

rmse_final = np.sqrt(float(hist['val_mse'].tail(1)))
print()
print("Final RMSE on val: ",round(rmse_final,3))
hist.iloc[-1:]


......................................................................................................................................................
......................................................................................................................................................
............................................................................................
Final RMSE on val:  0.332


,loss,accuracy,mse,val_loss,val_accuracy,val_mse,epoc
391,0.23033,0.894325,0.071354,0.370713,0.853881,0.11055,391


### Evaluation

In [185]:
mse, acc, _ = model.evaluate(X_test, Y_test)
rmse = np.sqrt(mse)
print("RMSE on test: ",round(rmse,3))
print("Accuracy on test: ",round(acc,3))

7/7 [==============================] - 0s 2ms/step - loss: 0.4718 - accuracy: 0.8493 - mse: 0.1192
RMSE on test:  0.687
Accuracy on test:  0.849


**without  normalization, the accuracy was 48%**